In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [23]:
from datetime import datetime

import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import LearningRateMonitor
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from era_data import TabletPeriodDataset, get_IDS
from era_model import SimpleCNN

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Hyperparameters

In [4]:
LR = 5e-5
EPOCHS = 5
BATCH_SIZE = 16
SUFFIX = '-vanillaCNN'
DATE = datetime.now().strftime("%B%d")
RUN_NAME_SUFFIX = '-preprocessed' # ''
IMG_DIR = 'output/images_preprocessed'
IDS = get_IDS(IMG_DIR=IMG_DIR)
print(len(IDS))
VERSION_NAME = f'period_clf_bs{BATCH_SIZE}_lr{LR}_{EPOCHS}epochs{SUFFIX}-{len(IDS)}_samples{RUN_NAME_SUFFIX}-{DATE}_1000test'
VERSION_NAME

94936


'period_clf_bs16_lr5e-05_5epochs-vanillaCNN-94936_samples-preprocessed-March28_1000test'

# Load data

In [5]:
#! du -h {IMG_DIR}

In [6]:
train_ids, test_ids = train_test_split(IDS, test_size=1000, random_state=0)
len(train_ids), len(test_ids)

(93936, 1000)

In [12]:
train_ids, val_ids = train_test_split(train_ids, test_size=1000, random_state=0)
len(train_ids), len(test_ids)

(92936, 1000)

In [13]:
ds_train = TabletPeriodDataset(IDS=train_ids, IMG_DIR=IMG_DIR)
ds_val = TabletPeriodDataset(IDS=val_ids, IMG_DIR=IMG_DIR)
ds_test = TabletPeriodDataset(IDS=test_ids, IMG_DIR=IMG_DIR)

Filtering 94936 IDS down to provided 92936...
Filtering 94936 IDS down to provided 1000...
Filtering 94936 IDS down to provided 1000...


In [15]:
def collate_fn(batch):
    data = torch.stack([torch.from_numpy(sample[1]) for sample in batch])
    labels = torch.tensor([sample[2] for sample in batch])

    return data, labels

In [16]:
dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE,collate_fn=collate_fn, shuffle=True, num_workers=4)
dl_val = DataLoader(ds_val, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False, num_workers=4)
dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False, num_workers=4)

In [17]:
# save model IDs so we can keep track of what data it was trained on
pd.Series(train_ids).to_csv(f'output/clf_ids/period-train-{VERSION_NAME}.csv', index=False, header=None)
pd.Series(val_ids).to_csv(f'output/clf_ids/period-val-{VERSION_NAME}.csv', index=False, header=None)
pd.Series(test_ids).to_csv(f'output/clf_ids/period-test-{VERSION_NAME}.csv', index=False, header=None)

In [18]:
num_classes = len(TabletPeriodDataset.PERIOD_INDICES) + 2
num_classes

24

In [19]:
model = SimpleCNN(num_classes=num_classes, learning_rate=LR)

In [24]:
logger = pl.loggers.TensorBoardLogger(
    save_dir='.',
    name='lightning_logs',
    version=VERSION_NAME
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator='gpu',
    devices='auto',
    callbacks=[lr_monitor],
    logger=logger
)


/home/kapond/.conda/envs/dani_torch/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kapond/.conda/envs/dani_torch/lib/python3.11/s ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [25]:
print('Logs to:', VERSION_NAME)

Logs to: period_clf_bs16_lr5e-05_5epochs-vanillaCNN-94936_samples-preprocessed-March28_1000test


In [26]:
trainer.fit(model, dl_train, dl_val)

/home/kapond/.conda/envs/dani_torch/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kapond/.conda/envs/dani_torch/lib/python3.11/s ...
  rank_zero_warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 10.76 GiB total capacity; 1.49 MiB already allocated; 885.56 MiB free; 22.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF